In [3]:
import pyodbc
import pandas.io.sql
import pandas as pd

In [2]:
sqlConnection = pyodbc.connect('DRIVER={SQL Server};server=TARDIS\\TARDIS;database=ScholarshipAwardingProcess;trusted_connection=true')

In [6]:
sql="Select * from Algorithms"
algorithms = pandas.io.sql.read_sql(sql, sqlConnection)
algorithms

,AlgorithmId,AlgorithmName,AlgorithmDescription
0,1,Merit Only,Only use top applicant for scholarship
1,2,Merit Only Disqualify After Exceeding Maximum,Use top but top out a single person
2,3,Merit Only Awarding Can’t Exceed Maximum\r\n,Use top but don't let go over certain amount
3,4,Maximum One Award Per Applicant,Only allow an applicant to have a single schol...
4,5,Split with all qualified applicants\r\n,Split award with all qualified applicants
5,6,Split with minimum qualified applicants\r\n,Split award with applicants up to a certain nu...
6,7,Split with minimum amount given\r\n,Split evenly up to split doesn't bring award a...


In [ ]:
pandas.io.sql.read_sql("")